In [3]:
import numpy as np
import pandas as pd
import os
import tqdm
import pickle
from tqdm.notebook import tqdm

from functools import lru_cache

from snorkel.labeling import labeling_function
from snorkel.labeling import LabelingFunction
from snorkel.labeling.model import LabelModel
from snorkel.labeling import PandasLFApplier

#from labeling_functions import get_all_lfs
from pruning_lfs import prune_lfs

import glob

In [4]:
all_lfs = prune_lfs()

/workspace/pruning_lfs.py:52: RuntimeWarning: invalid value encountered in scalar divide
  precision = cov_src/(cov_simp + cov_src)


In [6]:
len(all_lfs)

311

In [7]:
with open("workspace/datasets/final_combined_with_index.pkl", 'rb') as f:
    dataset = pickle.load(f)

In [8]:
ABSTAIN = -1
SIMPLE = 0
NOT_SIMPLE = 1
LOST_MEANING = 2

label_map = {-1: "ABSTAIN", 0: "SIMPLE", 1: "NOT_SIMPLE", 2: "LOST_MEANING"}

In [9]:
dataset['simplified_snt'] = dataset['simp']
dataset['source_snt'] = dataset['src']

In [10]:
label_path = "/workspace/datasets/3mlabels/"

In [11]:
if not os.path.isdir(label_path):
    os.mkdir(label_path)

In [12]:
def get_finished_batches(path = "/workspace/datasets/3mlabels/*"):
    label_paths = glob.glob(path)
    fin_batches = [int(path.split("_")[-1]) for path in label_paths if "labels_" in path]
    return set(fin_batches)

In [13]:
def save_used_lfs(all_lfs, path = "/workspace/datasets/3mlabels/used_lfs.pkl"):
    names = [lf.name for lf in all_lfs]
    pickle.dump(names, open(path, "wb"))

In [14]:
batch_size = 100
start = 0

save_used_lfs(all_lfs)

for i in tqdm(range(start, len(dataset), batch_size), position=1):
    if not i in get_finished_batches():
        try:
            applier = PandasLFApplier(all_lfs)
            labels = applier.apply(dataset[i:i+batch_size], progress_bar=True)
            
            pickle.dump(labels, open(f"/workspace/datasets/3mlabels/labels_{i}", "wb"))
            print(f"finished on {i}/{len(dataset)}")

        except Exception as e:
            print(f"something went wrong with batch {i}")
            print(e)

  0%|          | 0/150349 [00:00<?, ?it/s]

100%|██████████| 20/20 [00:23<00:00,  1.16s/it]


finished on 440/3006971


100%|██████████| 20/20 [00:23<00:00,  1.18s/it]


finished on 460/3006971


...

finished on 480/3006971


100%|██████████| 20/20 [00:23<00:00,  1.19s/it]


finished on 500/3006971


 15%|█▌        | 3/20 [00:03<00:19,  1.15s/it]


KeyboardInterrupt: 